<a href="https://colab.research.google.com/github/fahdleroi/Chatbot/blob/main/chatbott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tflearn 

     |████████████████████████████████| 112kB 12.2MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp36-none-any.whl size=127302 sha256=cee0ccf0fb0d1eae8b1e129e8a013ddad3db57d7a2299d00973bc4217a557361
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [2]:
import nltk   #Natural Language Toolkit
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#Stemmers remove morphological affixes from words, leaving only the word stem.

import numpy     #array management
import tflearn
import tensorflow
import random
import json
import pickle


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#read data
with open("intents.json") as file:                #json:Il permet de représenter de l’information structurée  , Search!!
    data = json.load(file)             
    #output:like python             dictionnaire


# Nouvelle section

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
try:
    with open("data.pickle", "rb") as f:                             #save data here for future predictioon , we don't need to execute except(part)
        words, labels, training, output = pickle.load(f)   # serialization : converting data into a byte stream
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:      #loop in dict
        for pattern in intent["patterns"]: 
            wrds = nltk.word_tokenize(pattern)    #do it -> "do","it"
            words.extend(wrds)                    #list of all words
            docs_x.append(wrds)                     
            docs_y.append(intent["tag"])           #matrix of words each line contains a column of its words separeted

        if intent["tag"] not in labels:    #add tags
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]   #remove duplicate and ? 
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]    # _  like x or y : It's just another variable.recommander :when we don't use the variables later

    for x, doc in enumerate(docs_x):                #frequency , 
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:             #[0,0,1,1,0,0,0]
            if w in wrds:
                bag.append(1)        #1:exist
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)            

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

In [6]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [7]:

try:
    model.load("model.tflearn")
except:

    
    net = tflearn.input_data(shape=[None, len(training[0])]) 
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation="softmax")   
    net = tflearn.regression(net)

    model = tflearn.DNN(net)# update
    

    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


#bag of words for prediction
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)     
    s_words = [stemmer.stem(word.lower()) for word in s_words]     

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)



Training Step: 13999  | total loss: 0.00268 | time: 0.092s
| Adam | epoch: 1000 | loss: 0.00268 - acc: 0.9999 -- iter: 104/110
Training Step: 14000  | total loss: 0.00247 | time: 0.097s
| Adam | epoch: 1000 | loss: 0.00247 - acc: 0.9999 -- iter: 110/110
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [8]:


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        #print(tag)

        #repondre ssi  >0,7 not only take max ,meme que parfois le max est mauvais
        if results[results_index] > 0.7:   #70% confidence
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
            #print(responses)
        else:
            print("I didn't get that,try again please .")    
chat()


Start talking with the bot (type quit to stop)!
You: salt
je suis là pour aider les etudiants de l'ENISAS ,je suis a votre disposition 
You: bonsoir
je suis là pour aider les etudiants de l'ENISAS ,je suis a votre disposition 
You: prix insc
I didn't get that,try again please .
You: prix inscript
I didn't get that,try again please .
You: prix inscription
I didn't get that,try again please .


KeyboardInterrupt: ignored